# 한국어 개체명 인식 개요
이번 강의에서는 버트를 활용해서 한국어 개체명 인식을 다뤄보고자 합니다. 
![Imgur](https://i.imgur.com/PDdTLjy.png) 
데이터는 https://github.com/naver/nlp-challenge/ 에서 받아왔습니다.

개체명 추출 리더보드에서 제공되는 코퍼스는 문장에 나타난 개체명을 14개 분류 카테고리로 주석 작업이 되어있습니다.

![Imgur](https://i.imgur.com/it4uTE3.png)

문장을 입력하면 다음과 같은 형식으로 개체명이 분류됩니다.  

![Imgur](https://i.imgur.com/RpuZf6R.png)




# 목차
이번 실습은 <b>1) 네이버 개체명 인식 데이터 불러오기 및 전처리 2) BERT 인풋 만들기 3) 버트를 활용한 개체명 인식 모델 만들기 4) 훈련 및 성능 검증 5) 실제 데이터로 실습하기</b>로 구성되어 있습니다.

# BERT를 활용하여 한국어 개체명 인식기 만들기

## 개체명 인식 데이터 불러오기 및 전처리

개체명 인식을 위한 데이터를 다운 받습니다.

In [ ]:
!wget https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data

--2022-05-18 08:41:53--  https://github.com/naver/nlp-challenge/raw/master/missions/ner/data/train/train_data
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data [following]
--2022-05-18 08:41:54--  https://raw.githubusercontent.com/naver/nlp-challenge/master/missions/ner/data/train/train_data
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16945023 (16M) [text/plain]
Saving to: ‘train_data’

train_data          100%[===================>]  16.16M  --.-KB/s    in 0.1s    

2022-05-18 08:41:54 (126 MB/s) - ‘train_data’ saved [16945023/16945

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash #Mecab 다운

--2022-05-18 08:41:55--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-05-18 08:41:55--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc80d3a7831398a0a16900bd1b8c.dl.dropboxusercontent.com/cd/0/get/BleDPibt4gax-giP4srySa6kRs31GUIN3gZqeoY6zPbwB4Kkw8cRr3XGPolkPih7jW15osqYDas4nv9EvDfzkNFF_4o9aYkvrPtJX4QRNYL_y4omSEhN-tEDuvawBH3AIxpbbxzCLEcMy24P1s7Tf-l64K87Oiv3JZyNzFe-aGnfUOFHqTqbQSWJKuyEc0-pFiE/file?dl=1# [following]
--2022-05-18 08:41:55--  https://uc80d3a7831398a0a16900bd1b8c.dl.dropboxusercontent.com/cd/0/get/

분석에 필요한 모듈들을 임포트 합니다.

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install sacremoses
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
from tqdm import tqdm
import os
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 59.3 MB/s 
     |████████████████████████████████| 6.6 MB 40.2 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 880 kB 5.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e4a6d38bc45256e2a6159967f8ea6390bd2c40e04ab9fbe7df9d35693448c084
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


train 데이터를 불러 오겠습니다.

In [ ]:
train = pd.read_csv("train_data", names=['src', 'tar'], sep="\t")
train = train.reset_index()
train

,index,src,tar
0,1,비토리오,PER_B
1,2,양일,DAT_B
2,3,만에,-
3,4,영사관,ORG_B
4,5,감호,CVL_B
...,...,...,...
769060,2,어째,-
769061,3,뭔가,-
769062,4,수상쩍은,-
769063,5,좌담,-


train 데이터에 마침표가 이상한 것들이 많아서 확실하게 .으로 수정해 주겠습니다.

In [ ]:
train['src'] = train['src'].str.replace("．", ".", regex=False)
train

,index,src,tar
0,1,비토리오,PER_B
1,2,양일,DAT_B
2,3,만에,-
3,4,영사관,ORG_B
4,5,감호,CVL_B
...,...,...,...
769060,2,어째,-
769061,3,뭔가,-
769062,4,수상쩍은,-
769063,5,좌담,-


In [ ]:
train.loc[train['src']=='.']

,index,src,tar
15,6,.,-
30,15,.,-
40,10,.,-
77,24,.,-
96,19,.,-
...,...,...,...
769013,11,.,-
769036,23,.,-
769053,17,.,-
769058,5,.,-


데이터를 전처리 해주겠습니다.  
한글, 영어, 소문자, 대문자, . 이외의 단어들을 모두 제거하겠습니다.

In [ ]:
train['src'] = train['src'].astype(str)
train['tar'] = train['tar'].astype(str)

train['src'] = train['src'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', "", regex=True)#ㄱ-ㅣ가-힣0-9a-zA-Z로 시작하지 않는게 한번 이상 있으면, 공백으로 바꾼다

데이터를 리스트 형식으로 변환합니다.

In [ ]:
data = [list(x) for x in train[['index', 'src', 'tar']].to_numpy()]
data

[[1, '비토리오', 'PER_B'],
 [2, '양일', 'DAT_B'],
 [3, '만에', '-'],
 [4, '영사관', 'ORG_B'],
 [5, '감호', 'CVL_B'],
 [6, '용퇴', '-'],
 [7, '항룡', '-'],
 [8, '압력설', '-'],
 [9, '의심만', '-'],
 [10, '가율', '-'],
 [1, '이', '-'],
 [2, '음경동맥의', '-'],
 [3, '직경이', '-'],
 [4, '8', 'NUM_B'],
 [5, '19mm입니다', 'NUM_B'],
 [6, '.', '-'],
 [1, '9세이브로', 'NUM_B'],
 [2, '구완', '-'],
 [3, '30위인', 'NUM_B'],
 [4, 'LG', 'ORG_B'],
 [5, '박찬형은', 'PER_B'],
 [6, '평균자책점이', '-'],
 [7, '16.45로', 'NUM_B'],
 [8, '준수한', '-'],
 [9, '편이지만', '-'],
 [10, '22이닝', 'NUM_B'],
 [11, '동안', '-'],
 [12, '피홈런이', '-'],
 [13, '31개나', 'NUM_B'],
 [14, '된다', '-'],
 [15, '.', '-'],
 [1, '7승', 'NUM_B'],
 [2, '25패는', 'NUM_B'],
 [3, '상트페테르부르크가', 'LOC_B'],
 [4, '역대', '-'],
 [5, '월드리그에', 'EVT_B'],
 [6, '출진한', '-'],
 [7, '분별', '-'],
 [8, '최선의', '-'],
 [9, '성적이다', '-'],
 [10, '.', '-'],
 [1, '', '-'],
 [2, '퍼거슨', 'PER_B'],
 [3, '씨족의', 'CVL_B'],
 [4, '꾀', '-'],
 [1, '유로2008', 'EVT_B'],
 [2, '공인구가', 'CVL_B'],
 [3, '변할', '-'],
 [4, '기록', '-'],
 [5, '시정조치는', 'CVL_B'

데이터를 잘 보면 (인덱스, 단어, 개체) 로 이루어 진 것을 알 수 있습니다.  
인덱스가 1,2,3,4,5.. 이렇게 이어지다가 다시 1,2,3,4, 로 바뀌는데 숫자가 바뀌기 전까지가 한 문장을 의미합니다.

In [ ]:
print(data[:20])

[[1, '비토리오', 'PER_B'], [2, '양일', 'DAT_B'], [3, '만에', '-'], [4, '영사관', 'ORG_B'], [5, '감호', 'CVL_B'], [6, '용퇴', '-'], [7, '항룡', '-'], [8, '압력설', '-'], [9, '의심만', '-'], [10, '가율', '-'], [1, '이', '-'], [2, '음경동맥의', '-'], [3, '직경이', '-'], [4, '8', 'NUM_B'], [5, '19mm입니다', 'NUM_B'], [6, '.', '-'], [1, '9세이브로', 'NUM_B'], [2, '구완', '-'], [3, '30위인', 'NUM_B'], [4, 'LG', 'ORG_B']]


라벨들을 추출하고, 딕셔너리 형식으로 저장하도록 하겠습니다.

In [ ]:
label = train['tar'].unique().tolist()
label_dict = {word:i for i, word in enumerate(label)}
label_dict.update({"[PAD]":len(label_dict)})
index_to_ner = {i:j for j, i in label_dict.items()}

In [ ]:
print(label_dict)

{'PER_B': 0, 'DAT_B': 1, '-': 2, 'ORG_B': 3, 'CVL_B': 4, 'NUM_B': 5, 'LOC_B': 6, 'EVT_B': 7, 'TRM_B': 8, 'TRM_I': 9, 'EVT_I': 10, 'PER_I': 11, 'CVL_I': 12, 'NUM_I': 13, 'TIM_B': 14, 'TIM_I': 15, 'ORG_I': 16, 'DAT_I': 17, 'ANM_B': 18, 'MAT_B': 19, 'MAT_I': 20, 'AFW_B': 21, 'FLD_B': 22, 'LOC_I': 23, 'AFW_I': 24, 'PLT_B': 25, 'FLD_I': 26, 'ANM_I': 27, 'PLT_I': 28, '[PAD]': 29}


In [ ]:
print(index_to_ner)

{0: 'PER_B', 1: 'DAT_B', 2: '-', 3: 'ORG_B', 4: 'CVL_B', 5: 'NUM_B', 6: 'LOC_B', 7: 'EVT_B', 8: 'TRM_B', 9: 'TRM_I', 10: 'EVT_I', 11: 'PER_I', 12: 'CVL_I', 13: 'NUM_I', 14: 'TIM_B', 15: 'TIM_I', 16: 'ORG_I', 17: 'DAT_I', 18: 'ANM_B', 19: 'MAT_B', 20: 'MAT_I', 21: 'AFW_B', 22: 'FLD_B', 23: 'LOC_I', 24: 'AFW_I', 25: 'PLT_B', 26: 'FLD_I', 27: 'ANM_I', 28: 'PLT_I', 29: '[PAD]'}


데이터를 문장들과 개체들로 분리합니다.  
tups[0], tups[1],... 에 각각의 문장에 해당하는 단어와 개체 번호가 저장이 되게 됩니다.

In [ ]:
tups = []
temp_tup = []
temp_tup.append(data[0][1:])
sentences = []
targets = []
for i, j, k in data:
  
  if i != 1:
    temp_tup.append([j,label_dict[k]])
  if i == 1:
    if len(temp_tup) != 0:
      tups.append(temp_tup)
      temp_tup = []
      temp_tup.append([j,label_dict[k]])
# tups[0]는 첫번째 문장에 있는 단어들+개체번호, tups[1]은 두번째 문장에 있는 단어들+개체번호, ...
tups.pop(0)

[['비토리오', 'PER_B']]

In [ ]:
print(tups[0], tups[1])

[['비토리오', 0], ['양일', 1], ['만에', 2], ['영사관', 3], ['감호', 4], ['용퇴', 2], ['항룡', 2], ['압력설', 2], ['의심만', 2], ['가율', 2]] [['이', 2], ['음경동맥의', 2], ['직경이', 2], ['8', 5], ['19mm입니다', 5], ['.', 2]]


tups를 보면 [(단어, 개체), (단어, 개체), (단어, 개체)]의 형식으로 저장이 되어 있는데, 이거를 (단어, 단어, 단어, 단어), (개체, 개체, 개체, 개체) 형식으로 변환하도록 하겠습니다.


In [ ]:
sentences = []
targets = []
for tup in tups:
  sentence = []
  target = []
  sentence.append("[CLS]")
  target.append(label_dict['-'])
  for i, j in tup:
    sentence.append(i)
    target.append(j)
  sentence.append("[SEP]")
  target.append(label_dict['-'])
  sentences.append(sentence)
  targets.append(target)

In [ ]:
sentences[0]

['[CLS]',
 '비토리오',
 '양일',
 '만에',
 '영사관',
 '감호',
 '용퇴',
 '항룡',
 '압력설',
 '의심만',
 '가율',
 '[SEP]']

In [ ]:
targets[0]

[2, 0, 1, 2, 3, 4, 2, 2, 2, 2, 2, 2]

## 버트 인풋 만들기

구글의 multilinguial-bert를 활용하도록 하겠습니다.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpb65aogm7


Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
creating metadata file for /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpceyh4bsr


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://hug

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
creating metadata file for /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidde

In [ ]:
tokenizer.tokenize("안녕하세요 저는 서울시립대학교 전자전기컴퓨터공학부 최윤석입니다")

['안',
 '##녕',
 '##하',
 '##세',
 '##요',
 '저',
 '##는',
 '서울',
 '##시',
 '##립',
 '##대학교',
 '전',
 '##자',
 '##전',
 '##기',
 '##컴',
 '##퓨',
 '##터',
 '##공',
 '##학',
 '##부',
 '최',
 '##윤',
 '##석',
 '##입',
 '##니다']

여기서부터가 중요한데, 문장을 토크나이징 하고 개체(target)을 토크나이징 한 문장에 맞추도록 하겠습니다.  
문장 "대한민국 만세." 는 사실 (대한민국, 개체1), (만세., 개체2) 을 가지고 있는데 토크나이징을 하면 '▁대한민국', '▁만', '세', '.' 로 토크나이징이 됩니다.  
여기서 그렇다면 ( ▁대한민국, 개체1) , (▁만, 개체2), (세, 개체2), (., 개체 2) 와 같은 방식으로 각 개체를 부여해주어야 합니다.

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []

  for word, label in zip(sentence, text_labels):

    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    tokenized_sentence.extend(tokenized_word)
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [
                              tokenize_and_preserve_labels(sent, labs)
                              for sent, labs in zip(sentences, targets)]

In [ ]:
print(tokenized_texts_and_labels[:2])
# [(문장, 개체들), (문장, 개체들),...] 형식으로 저장되어 있음.

[(['[CLS]', '비', '##토', '##리', '##오', '양', '##일', '만', '##에', '영', '##사', '##관', '감', '##호', '용', '##퇴', '항', '##룡', '압', '##력', '##설', '의', '##심', '##만', '가', '##율', '[SEP]'], [2, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 3, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]), (['[CLS]', '이', '음', '##경', '##동', '##맥', '##의', '직', '##경', '##이', '8', '19', '##mm', '##입', '##니다', '.', '[SEP]'], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2])]


(문장, 개체들), (문장, 개체들) 을 [문장, 문장, 문장, ...] , [개체들, 개체들 개체들,,,,]로 분리해주도록 하겠습니다.

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
tokenized_texts[1]

['[CLS]',
 '이',
 '음',
 '##경',
 '##동',
 '##맥',
 '##의',
 '직',
 '##경',
 '##이',
 '8',
 '19',
 '##mm',
 '##입',
 '##니다',
 '.',
 '[SEP]']

In [ ]:
labels[1]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 2, 2]

문장의 길이가 상위 2.5%(88) 인 지점을 기준으로 문장의 길이를 정하도록 하겠습니다.  
만약 문장의 길이가 88보다 크면 문장이 잘리게 되고, 길이가 88보다 작다면 패딩이 되어 모든 문장의 길이가 88로 정해지게 됩니다.

In [ ]:
print(np.quantile(np.array([len(x) for x in tokenized_texts]), 0.975))
max_len = 100
bs = 1

96.0


버트에 인풋으로 들어갈 train 데이터를 만들도록 하겠습니다.  
버트 인풋으로는   
input_ids : 문장이 토크나이즈 된 것이 숫자로 바뀐 것,   
attention_masks : 문장이 토크나이즈 된 것 중에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 마스킹  
[input_ids, attention_masks]가 인풋으로 들어갑니다.

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype = "int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")

In [ ]:
input_ids[1]

array([   101,   9638,   9634,  31720,  18778, 118915,  10459,   9707,
        31720,  10739,    129,  10270,  17525,  58303,  48345,    119,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0])

정답에 해당하는 개체들을 만들어 보겠습니다.  
패딩에 해당하는 부분은 label_dict([PAD])(29)가 들어가게 되겠습니다.

In [ ]:
tags = pad_sequences([lab for lab in labels], maxlen=max_len, value=label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [ ]:
tags[1]

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  5,  5,  5,  5,  5,  2,  2,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29])

어텐션 마스크를 만들어 주겠습니다.

In [ ]:
attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in input_ids])
attention_masks[1]
# 어텐션 마스크란 BERT가 어텐션 연산을 할때, 불필요하게 패딩토큰에 대해 어텐션을 하지 않게, 실제 토큰과 패딩을 구분해준다 : 실제 - 1, 패딩 - 0

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

train 데이터에서 10% 만큼을 validation 데이터로 분리해 주겠습니다.

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)

In [ ]:
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

## 개체명 인식 모델 만들기

In [ ]:
# TPU 작동을 위해 실행
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
# TPU 초기화 하는 코드

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.97.28.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.97.28.130:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(30, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)
  
  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
  
  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

## 훈련 및 성능 검증

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  nr_model = create_model()
  nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
creating metadata file for /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
loading weights file https://huggingface.co/bert-base-multilingual-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
Loading PyTorch weights from /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
PyTorch checkpoint contains 270,378,749 parameters


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3


58968/58968 [==============================] - 4954s 83ms/step - loss: 0.1838 - sparse_categorical_accuracy: 0.9464 - val_loss: 0.1438 - val_sparse_categorical_accuracy: 0.9570
Epoch 2/3
58968/58968 [==============================] - 5032s 85ms/step - loss: 0.1282 - sparse_categorical_accuracy: 0.9614 - val_loss: 0.1345 - val_sparse_categorical_accuracy: 0.9600
Epoch 3/3
58968/58968 [==============================] - 4986s 85ms/step - loss: 0.1045 - sparse_categorical_accuracy: 0.9683 - val_loss: 0.1339 - val_sparse_categorical_accuracy: 0.9621


In [ ]:
# 만약 TPU를 사용하지 않고 GPU를 사용한다면
#nr_model = create_model()
#nr_model.fit([tr_inputs, tr_masks], tr_tags, validation_data=([val_inputs, val_masks], val_tags), epochs=3, shuffle=False, batch_size=bs)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
y_predicted = nr_model.predict([val_inputs, val_masks])

In [ ]:
f_label = [i for i, j in label_dict.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(val_tags).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(np.argmax(y_predicted, axis=2)).astype(int).tolist()] #axis 2일경우 z축 기준
f_label.remove("[PAD]")

각 개체별 f1 score를 측정하도록 하겠습니다.  
참고로 micro avg는 전체개수로 f1 score을 측정한 것이며,
macro avg는 각 f1-score를 평균을 낸 것이고,  
weighted avg는 각 개체별 f1 score*support비율을 해서 더한 것입니다.




In [ ]:
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

       PER_B       0.87      0.79      0.83     11582
       DAT_B       0.92      0.82      0.87      4438
           -       0.94      0.96      0.95    138244
       ORG_B       0.79      0.85      0.82     13737
       CVL_B       0.75      0.82      0.78     15975
       NUM_B       0.93      0.92      0.93     11267
       LOC_B       0.78      0.78      0.78      5958
       EVT_B       0.76      0.78      0.77      3917
       TRM_B       0.80      0.70      0.75      6102
       TRM_I       0.52      0.27      0.36       690
       EVT_I       0.82      0.60      0.69      1855
       PER_I       0.62      0.76      0.68      1592
       CVL_I       0.61      0.26      0.36       881
       NUM_I       0.73      0.74      0.73      1548
       TIM_B       0.75      0.93      0.83       630
       TIM_I       0.93      0.67      0.78       219
       ORG_I       0.87      0.32      0.47      1508
       DAT_I       0.87    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision(정밀도) = TP/(TP+FP) -> Positive라 답한것중 진짜 Positive일 확률\
recall(재현율) = TP/(TP+FN) -> 실제로 Positive 인것을 모델이 Positive라 인식한 확률\
f1-score : precision과 recall 의 조화평균 = 2 * precision * recall / (precision+recall)



# 실제 데이터로 실습하기

In [ ]:
def ner_inference(test_sentence):
  

  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(index_to_ner[label_idx])
  
  #user_dict={}
  for token, label in zip(new_tokens, new_labels):
      user_dict.update({token:label})
      print("{}\t{}".format(label, token))
  
  print("\n") 

In [ ]:
user_dict={}
ner_inference("문재인 대통령은 1953년 1월 24일 경상남도 거제시에서 아버지 문용형과 어머니 강한옥 사이에서 둘째(장남)로 태어났다.")
sorted_dict = sorted(user_dict.items(), key = lambda item: item[1])
print(sorted_dict)

PER_B	문
PER_B	재
PER_B	인
CVL_B	대통령
CVL_B	은
DAT_B	1953
DAT_B	년
DAT_I	1월
DAT_I	24일
LOC_B	경
LOC_B	상
LOC_B	남도
LOC_B	거
LOC_B	제
LOC_B	시
LOC_B	에서
CVL_B	아버지
PER_B	문
PER_B	용
PER_B	형
PER_B	과
CVL_B	어머니
PER_B	강
PER_B	한
PER_B	옥
-	사이에
-	서
NUM_B	둘
NUM_B	째
-	(
-	장
-	남
-	)
-	로
-	태어났다
-	.


[('사이에', '-'), ('서', '-'), ('(', '-'), ('장', '-'), ('남', '-'), (')', '-'), ('로', '-'), ('태어났다', '-'), ('.', '-'), ('대통령', 'CVL_B'), ('은', 'CVL_B'), ('아버지', 'CVL_B'), ('어머니', 'CVL_B'), ('1953', 'DAT_B'), ('년', 'DAT_B'), ('1월', 'DAT_I'), ('24일', 'DAT_I'), ('경', 'LOC_B'), ('상', 'LOC_B'), ('남도', 'LOC_B'), ('거', 'LOC_B'), ('제', 'LOC_B'), ('시', 'LOC_B'), ('에서', 'LOC_B'), ('둘', 'NUM_B'), ('째', 'NUM_B'), ('문', 'PER_B'), ('재', 'PER_B'), ('인', 'PER_B'), ('용', 'PER_B'), ('형', 'PER_B'), ('과', 'PER_B'), ('강', 'PER_B'), ('한', 'PER_B'), ('옥', 'PER_B')]


In [ ]:
my_list=[]
for i in range(len(sorted_dict)):
    my_list.append(sorted_dict[i][0])
my_list

['사이에',
 '서',
 '(',
 '장',
 '남',
 ')',
 '로',
 '태어났다',
 '.',
 '대통령',
 '은',
 '아버지',
 '어머니',
 '1953',
 '년',
 '1월',
 '24일',
 '경',
 '상',
 '남도',
 '거',
 '제',
 '시',
 '에서',
 '둘',
 '째',
 '문',
 '재',
 '인',
 '용',
 '형',
 '과',
 '강',
 '한',
 '옥']

In [ ]:
ner_inference("안녕하세요 저는 서울시립대학교 전자전기컴퓨터공학부 최윤석입니다.")

-	안
-	녕
-	하
-	세
-	요
-	저
-	는
ORG_B	서울
ORG_B	시
ORG_B	립
ORG_B	대학교
ORG_B	전
ORG_B	자
ORG_B	전
ORG_B	기
ORG_B	컴
ORG_B	퓨
ORG_B	터
ORG_B	공
ORG_B	학
ORG_B	부
PER_B	최
PER_B	윤
PER_B	석
PER_B	입
PER_B	니다
-	.




In [ ]:
ner_inference("인공지능의 역사는 20세기 초반에서 더 거슬러 올라가보면 이미 17~18세기부터 태동하고 있었지만 이때는 인공지능 그 자체보다는 뇌와 마음의 관계에 관한 철학적인 논쟁 수준에 머무르고 있었다. 그럴 수 밖에 없는 것이 당시에는 인간의 뇌 말고는 정보처리기계가 존재하지 않았기 때문이다. ")

-	인
-	공
-	지
-	능
-	의
-	역
-	사는
NUM_B	20
NUM_B	세기
-	초
-	반
-	에서
-	더
-	거
-	슬
-	러
-	올
-	라
-	가
-	보
-	면
-	이미
NUM_B	17
NUM_B	~
NUM_B	18
NUM_B	세기
NUM_B	부터
-	태
-	동
-	하고
-	있
-	었지만
-	이때
-	는
-	인
-	공
-	지
-	능
-	그
-	자
-	체
-	보다
-	는
-	뇌
-	와
-	마
-	음
-	의
-	관
-	계에
-	관한
-	철
-	학적
-	인
-	논
-	쟁
-	수
-	준
-	에
-	머
-	무
-	르고
-	있었다
-	.
-	그
-	럴
-	수
-	밖
-	에
-	없는
-	것이
-	당시
-	에는
-	인
-	간의
-	뇌
-	말
-	고
-	는
-	정
-	보
-	처
-	리
-	기
-	계
-	가
-	존
-	재
-	하지
-	않
-	았
-	기
-	때문이다
-	.




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename_news = '/content/drive/MyDrive/NLP/20190905_20200322_by500.csv'
data_news = pd.read_csv(filename_news).astype(str)
data_news

ParserError: ignored

In [ ]:
import itertools
Temp_news=[]
Temp_news.append(data_news['CONTENT'])
Data = list(itertools.chain.from_iterable(Temp_news))

In [ ]:
Data[0]

In [ ]:
ner_inference(Data[0])

In [ ]:
import random
YS=[]
for i in range(30):
    YS.append(random.randrange(0,99999))


input_data=""
for i in YS:
    input_data = input_data + Data[i]

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

In [ ]:
input_pos = mecab.pos(input_data)
input_pos

In [ ]:
extracted_NNP = [ word for word, tag in input_pos if tag in ["NNP"] ]
extracted_NNP = list(set(extracted_NNP))
extracted_NNP

In [ ]:
for i in range(20):
    ner_inference(extracted_NNP[i])


In [ ]:
!pip install node2vec
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from node2vec import Node2Vec
from sklearn.manifold import TSNE
import numpy as np
from numpy import dot
from numpy.linalg import norm
import seaborn as sns

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
# 기존 node2vec 모델 (2018년 1월 1일부터 2020년 1월 1일까지 학습시킨 모델)
dir_model = '/content/drive/MyDrive/NLP/20180101_20200101_weightedN2V.bin'
n2vmodel = KeyedVectors.load_word2vec_format(dir_model)

In [ ]:
vocab = [ word for word in extracted_NNP if word in n2vmodel]

In [ ]:
vocab_vector = []
for word in vocab:
    vocab_vector.append(n2vmodel[word])

In [ ]:
len_v=len(vocab_vector)
len(vocab_vector)

In [ ]:
zero_data = np.zeros(shape=(len_v,len_v))
cos_array = pd.DataFrame(zero_data, columns=vocab)
cos_array.index=[vocab]

for i in range(len_v):
    for j in range(len_v):
        cos_array.iloc[i,j]=cos_sim(vocab_vector[i],vocab_vector[j])
cos_array


In [ ]:
def cos_desc(obj):
    print(cos_array.loc[:,obj].sort_values(ascending=False))

In [ ]:
cos_desc('한반도')

In [ ]:
# 한글폰트 사용 in colab
%matplotlib inline  

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
from sklearn.manifold import TSNE 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import pandas as pd 
from gensim.models import KeyedVectors 

l_rate=len(vocab_vector)
perp=len(vocab_vector)/20+5

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처 
mpl.rcParams['axes.unicode_minus'] = False 
mpl.rcParams['font.family'] = 'NanumBarunGothic'

tsne = TSNE(n_components=2,verbose=1,learning_rate=l_rate,perplexity=perp, n_iter=1000, early_exaggeration=20) 
X = tsne.fit_transform(vocab_vector) 
df = pd.DataFrame(X, index=vocab, columns=['x', 'y'])
df.head()
fig = plt.figure() 
plt.title('embedding vector')
fig.set_size_inches(30, 20) 
ax = fig.add_subplot(1, 1, 1)

ax.scatter(df['x'], df['y'])
for word, pos in df.iterrows():
    ax.annotate(word, pos, fontsize=30)
plt.show()

In [ ]:
from keras.models import load_model

nr_model.save('ner_by100_batch1.h5')